In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install torch

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'torch'

In [0]:
url = '/content/drive/My Drive/IBM/dataset/train.csv'
df = pd.read_csv(url)

In [0]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [0]:
tokenized = df['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [8]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(13240, 171)

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [1]:
features = last_hidden_states[0][:,0,:].numpy()

NameError: ignored

In [0]:
labels = df['subtask_a']

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [0]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

In [0]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

In [0]:
lr_clf.score(test_features, test_labels)